In [6]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import cv2

physical_devices = tf.config.experimental.list_physical_devices('GPU')
if physical_devices:
  print('use: GPU')
else:
  print('use: CPU')

use: CPU


In [7]:
DATASET_DIR = "./data-augement/images-generate"
CATEGORIES = ["one", "two", "three", "four", "five", "six", "seven", "eight", "nine", "ten"]

In [8]:
training_data = []
def create_training_data() -> None:
  for category in CATEGORIES:
    path = os.path.join(DATASET_DIR, category)
    class_num = CATEGORIES.index(category)
    for img in os.listdir(path):
      try:
        img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
        training_data.append([img_array, class_num])
      except Exception as e:
        pass
      
create_training_data()
print(len(training_data))

20000


In [9]:
import random
random.shuffle(training_data)

In [49]:
input_shape = (28, 28, 1)
num_classes = 10

x_train = []
y_train = []

for features, label in training_data:
  x_train.append(features)
  y_train.append(label)

x_train = np.array(x_train).reshape(-1, 28, 28, 1)
x_train = x_train.astype("float32") / 255

y_train = np.array(y_train)
y_train = keras.utils.to_categorical(y_train, num_classes)

print(x_train.shape)
print(y_train.shape)

x_test = x_train[int(0.9 * 20000):,:]
y_test = y_train[int(0.9 * 20000):,:]
print(x_test.shape)
print(y_test.shape)

(20000, 28, 28, 1)
(20000, 10)
(2000, 28, 28, 1)
(2000, 10)


In [11]:
model = keras.Sequential(
  [
    keras.Input(shape=input_shape),
    layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation="softmax"),
  ]
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dropout (Dropout)           (None, 1600)              0

In [12]:
batch_size = 128
epochs = 20

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/20
141/141 [==============================] - 14s 89ms/step - loss: 1.6780 - accuracy: 0.4261 - val_loss: 0.9915 - val_accuracy: 0.7105
Epoch 2/20
141/141 [==============================] - 18s 129ms/step - loss: 0.8140 - accuracy: 0.7401 - val_loss: 0.4890 - val_accuracy: 0.8735
Epoch 3/20
141/141 [==============================] - 18s 131ms/step - loss: 0.5136 - accuracy: 0.8405 - val_loss: 0.3226 - val_accuracy: 0.9075
Epoch 4/20
141/141 [==============================] - 13s 90ms/step - loss: 0.3918 - accuracy: 0.8779 - val_loss: 0.2553 - val_accuracy: 0.9360
Epoch 5/20
141/141 [==============================] - 12s 86ms/step - loss: 0.3203 - accuracy: 0.9006 - val_loss: 0.1970 - val_accuracy: 0.9500
Epoch 6/20
141/141 [==============================] - 12s 87ms/step - loss: 0.2636 - accuracy: 0.9193 - val_loss: 0.1631 - val_accuracy: 0.9550
Epoch 7/20
141/141 [==============================] - 13s 90ms/step - loss: 0.2278 - accuracy: 0.9298 - val_loss: 0.1338 - val_accurac

In [51]:
score = model.evaluate(x_test, y_test)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

model.save('out/model_characters.h5')

63/63 [==============================] - 1s 9ms/step - loss: 0.0423 - accuracy: 0.9880
Test loss: 0.042286403477191925
Test accuracy: 0.9879999756813049
